In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error


In [ ]:
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('test.csv')

# Data Exploration

In [ ]:
train_data[{'Embarked', 'Survived'}].groupby('Embarked').sum()

In [ ]:
train_data[train_data.Embarked == 'S'].count()

In [ ]:
test_data.head()

In [ ]:
missing_val_count_by_column = (train_data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

In [ ]:
train_data = train_data.drop(['Cabin'], axis = 1)
test_data = test_data.drop(['Cabin'], axis = 1)


In [ ]:
train_data['Embarked'] = train_data['Embarked'].fillna('S')
test_data['Embarked'] = test_data['Embarked'].fillna('S')

In [ ]:
y = train_data.Survived
feature_colums = ['Pclass', 'Age', 'Sex', 'Parch', 'Embarked']
X = train_data[feature_colums]
X_test = test_data[feature_colums]

In [ ]:
X_test

In [ ]:
my_imputer = SimpleImputer()
X['Age'] = pd.DataFrame(my_imputer.fit_transform(np.array(X['Age']).reshape(-1,1)))
X_test['Age'] = pd.DataFrame(my_imputer.fit_transform(np.array(X_test['Age']).reshape(-1,1)))

In [ ]:
OH_columns = ['Sex', 'Embarked']

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_X = pd.DataFrame(OH_encoder.fit_transform(X[OH_columns]))
OH_X_test = pd.DataFrame(OH_encoder.fit_transform(X_test[OH_columns]))

In [ ]:
OH_X_train = pd.concat([X.drop(OH_columns, axis=1), OH_X], axis=1)
OH_X_test = pd.concat([X_test.drop(OH_columns, axis=1), OH_X_test], axis=1)

In [ ]:
OH_X_train

In [ ]:
OH_X_train = OH_X_train.rename(columns={0: 'Female', 1: 'Male', 2: 'Embarked_C', 3: 'Embarked_Q', 4: 'Embarked_S'})
OH_X_test = OH_X_test.rename(columns={0: 'Female', 1: 'Male', 2: 'Embarked_C', 3: 'Embarked_Q', 4: 'Embarked_S'})

In [ ]:
OH_X_test

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=1)

In [ ]:
scores = -1 * cross_val_score(model, OH_X_train, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')
scores

In [ ]:
test_prediction = model.predict(OH_X_test).round().astype(int)

In [ ]:
result = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': test_prediction})

In [ ]:
result

In [ ]:
result.to_csv('results.csv', index = None)